In [1]:
import os
import shutil
from collections import Counter
import numpy as np
import torch

from helpers import makedir
import find_nearest
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from preprocess import mean, std, preprocess_input_function
from settings import train_dir, test_dir, train_push_dir, train_batch_size, test_batch_size, train_push_batch_size
from settings import base_architecture, img_size, prototype_shape, num_classes, prototype_activation_function, add_on_layers_type
import train_and_test as tnt

In [3]:
def prune_prototypes(dataloader,
                     prototype_network_parallel,
                     nearest_train_patch_class_ids,
                     k,
                     prune_threshold,
                     preprocess_input_function,
                     original_model_dir,
                     epoch_number,
                     #model_name=None,
                     log=print,
                     copy_prototype_imgs=True):
    ### run global analysis
    #nearest_train_patch_class_ids = \
    #    find_nearest.find_k_nearest_patches_to_prototypes(dataloader=dataloader,
    #                                                      prototype_network_parallel=prototype_network_parallel,
    #                                                      k=k,
    #                                                      preprocess_input_function=preprocess_input_function,
    #                                                      full_save=False,
    #                                                      log=log)
    
    ### find prototypes to prune
    original_num_prototypes = prototype_network_parallel.module.num_prototypes
    
    prototypes_to_prune = []
    for j in range(prototype_network_parallel.module.num_prototypes):
        class_j = torch.argmax(prototype_network_parallel.module.prototype_class_identity[j]).item()
        nearest_train_patch_class_counts_j = Counter(nearest_train_patch_class_ids[j])
        
        # if no such element is in Counter, it will return 0
        if nearest_train_patch_class_counts_j[class_j] < prune_threshold:
            prototypes_to_prune.append(j)

    log('k = {}, prune_threshold = {}'.format(k, prune_threshold))
    log('{} prototypes will be pruned'.format(len(prototypes_to_prune)))

    ### bookkeeping of prototypes to be pruned
    class_of_prototypes_to_prune = \
        torch.argmax(
            prototype_network_parallel.module.prototype_class_identity[prototypes_to_prune],
            dim=1).numpy().reshape(-1, 1)
    prototypes_to_prune_np = np.array(prototypes_to_prune).reshape(-1, 1)
    prune_info = np.hstack((prototypes_to_prune_np, class_of_prototypes_to_prune))
    #makedir(os.path.join(original_model_dir, 'pruned_prototypes_epoch{}_k{}_pt{}'.format(epoch_number,
    #                                      k,
    #                                      prune_threshold)))
    #np.save(os.path.join(original_model_dir, 'pruned_prototypes_epoch{}_k{}_pt{}'.format(epoch_number,
    #                                      k,
    #                                      prune_threshold), 'prune_info.npy'),
    #        prune_info)

    ### prune prototypes
    prototype_network_parallel.module.prune_prototypes(prototypes_to_prune)
    return prototypes_to_prune

In [4]:
normalize = transforms.Normalize(mean=mean, std=std)

train_dataset = datasets.ImageFolder(
        train_push_dir,
        transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=80, shuffle=False,
    num_workers=1, pin_memory=False)

test_dataset = datasets.ImageFolder(
        test_dir,
        transforms.Compose([
            transforms.Resize(size=(img_size, img_size)),
            transforms.ToTensor(),
            normalize,
        ]))
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=test_batch_size, shuffle=False,
    num_workers=1, pin_memory=False)



In [18]:
#ppnet = torch.load(r'../saved_models/vgg19/004/100_7push0.7344.pth')
ppnet = torch.load(r'./joint_results/003/12_12push0.7411.pth')
ppnet_multi = torch.nn.DataParallel(ppnet).cuda()

In [6]:
nearest_train_patch_class_ids = find_nearest.find_k_nearest_patches_to_prototypes(dataloader=train_loader,
                                                          prototype_network_parallel=ppnet_multi,
                                                          k=10,
                                                          preprocess_input_function=preprocess_input_function,
                                                          full_save=False,
                                                          log=print)

find nearest patches
1.0
batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
batch 20
batch 21
batch 22
batch 23
batch 24
batch 25
batch 26
batch 27
batch 28
batch 29
batch 30
batch 31
batch 32
batch 33
batch 34
batch 35
batch 36
batch 37
batch 38
batch 39
batch 40
batch 41
batch 42
batch 43
batch 44
batch 45
batch 46
batch 47
batch 48
batch 49
batch 50
batch 51
batch 52
batch 53
batch 54
batch 55
batch 56
batch 57
batch 58
batch 59
batch 60
batch 61
batch 62
batch 63
batch 64
batch 65
batch 66
batch 67
batch 68
batch 69
batch 70
batch 71
batch 72
batch 73
batch 74
	find nearest patches time: 	160.27685618400574


In [19]:
prototypes_to_prune = prune_prototypes(train_loader, ppnet_multi, nearest_train_patch_class_ids, 10, 1, preprocess_input_function, original_model_dir=None, epoch_number=100, log=print, copy_prototype_imgs=False)

k = 10, prune_threshold = 1
64 prototypes will be pruned


In [20]:
print(prototypes_to_prune)

[5, 6, 18, 22, 46, 58, 96, 129, 130, 133, 145, 172, 189, 190, 244, 261, 263, 269, 298, 320, 321, 332, 367, 368, 399, 404, 410, 419, 433, 451, 454, 479, 484, 514, 529, 540, 554, 567, 600, 608, 655, 671, 680, 692, 718, 765, 789, 796, 801, 804, 813, 821, 833, 856, 873, 894, 921, 935, 939, 950, 953, 954, 978, 983]


In [21]:
acc = tnt.test(model=ppnet_multi, dataloader=test_loader, class_specific=True, log=print)

	test


58it [00:41,  1.38it/s]


	time: 	42.00515055656433
	cross ent: 	1.315489997637683
	cluster: 	0.05828927592213812
	separation:	0.06427808317901759
	avg separation:	1.1304978831061
	accu: 		73.47255781843286%
	l1: 		1202.0126953125
	p dist pair: 	2.1344339847564697


In [16]:
ppnet_multi.module.prototype_vectors.data.shape

torch.Size([846, 128, 1, 1])

In [22]:
torch.save(ppnet_multi.module, r'./A3C_results/joint_004_pruned_10_1_0.7347')